### Perpare resources

In [1]:
!pip install -U sagemaker

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(
    framework_version="1.2-1", role=role, instance_type="ml.m5.xlarge", instance_count=1)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Download Data

In [3]:
import pandas as pd

s3 = boto3.client("s3")
s3.download_file(
    "sagemaker-sample-data-{}".format(region),
    "processing/census/census-income.csv",
    "census-income.csv",
)
df = pd.read_csv("census-income.csv")
df.to_csv("dataset.csv")
df.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,...,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,income
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


### Prepare Processing script

In [4]:
%%writefile preprocessing.py
import pandas as pd
import os
from sklearn.model_selection import train_test_split

input_data_path = os.path.join("/opt/ml/processing/input", "dataset.csv")
df = pd.read_csv(input_data_path)
print("Shape of data is: ", df.shape)
train, test = train_test_split(df, test_size=0.2)
train, validation = train_test_split(train, test_size=0.2)

train_path = "/opt/ml/processing/output/train"
valid_path = "/opt/ml/processing/output/validation"
test_path = "/opt/ml/processing/output/test"

is_exist_train = os.path.exists(train_path)
is_exist_valid = os.path.exists(valid_path)
is_exist_test = os.path.exists(test_path)

try:
    if not is_exist_train:
        os.makedirs("/opt/ml/processing/output/train")
    if not is_exist_valid:
        os.makedirs("/opt/ml/processing/output/validation")
    if not is_exist_test:
        os.makedirs("/opt/ml/processing/output/test")
    print("Successfully created directories")
except Exception as e:
    # if the Processing call already create these directories ( or directory otherwise cannot be created)
    print(e)
    print("Could not make directories")
    pass

try:
    train.to_csv("/opt/ml/processing/output/train/train.csv")
    validation.to_csv("/opt/ml/processing/output/validation/validation.csv")
    test.to_csv("/opt/ml/processing/output/test/test.csv")
    print("Wrote files successfully")
except Exception as e:
    print("Failed to write the files")
    print(e)
    pass

print("Completed running the processing job")
    

Overwriting preprocessing.py


### Run Processing job
Run the processing job, specifying the script name, input file, and output file

In [5]:
%%capture output
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="preprocessing.py",
    # arguments = ["arg1", "arg2"] # Arguments can optionally be specified here
    inputs=[ProcessingInput(source="dataset.csv", destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/train"),
        ProcessingOutput(source="/opt/ml/processing/output/validation"),
        ProcessingOutput(source="/opt/ml/processing/output/test"),
    ],
)

In [6]:
print(output)
job_name = str(output).split("\n")[1].split(" ")[-1]

.............................Shape of data is:  (199523, 43)
Successfully created directories
Wrote files successfully
Completed running the processing job




In [8]:
import boto3

s3_client = boto3.client("s3")
default_bucket = sagemaker.Session().default_bucket()
for i in range(1,4):
    prefix = s3_client.list_objects(Bucket=default_bucket, Prefix="sagemaker-scikit-learn")[
        "Contents"
    ][-i]["Key"]
    print("s3://" + default_bucket + "/" + prefix)

s3://sagemaker-ap-northeast-2-532805286864/sagemaker-scikit-learn-2024-04-03-02-25-32-450/output/output-3/test.csv
s3://sagemaker-ap-northeast-2-532805286864/sagemaker-scikit-learn-2024-04-03-02-25-32-450/output/output-2/validation.csv
s3://sagemaker-ap-northeast-2-532805286864/sagemaker-scikit-learn-2024-04-03-02-25-32-450/output/output-1/train.csv
